In [ ]:
import numpy as np
import glob
import pandas
from bokeh.io import *
from bokeh.plotting import figure, show, output_file, ColumnDataSource

df = pandas.DataFrame()
for f in glob.glob("C:\\Users\\aelek\\source\\repos\\FantaSeaViz\\data\\2017-18\\gws\\*"):
    dftmp = pandas.read_csv(f, encoding='ansi')   
    dftmp['name'] = dftmp['name'].str.replace('_', ' ')
    dftmp = dftmp[['name', 'value', 'total_points']]
    df = df.append(dftmp)

# group by player
res = df.groupby(['name']).median()

# calculate ratio
res['value'] = res['value']/10
res['vpc_ratio'] = res['total_points'] / res['value']
res = res[(res['total_points'] > 0) & (res['vpc_ratio'] > 0)]
res = res.sort_values(['vpc_ratio'], ascending=False)
res['name'] = res.index.values

x = res['value']
x = x.values
y = res['total_points'].values
names = res.index.values
vpc_ratio = res['vpc_ratio'].values
fill_alpha = np.full((len(res['vpc_ratio']), 1), 0.6)

In [242]:
source = ColumnDataSource(data=dict(
    x=x,
    y=y,
    names=names,
    vpc_ratio=vpc_ratio,
    circle_size=vpc_ratio/4,
    fill_alpha=fill_alpha))

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

p = figure(tools=TOOLS)
p.hover.tooltips = [
    ("name", "@names"),
    ("value per cost", "@vpc_ratio"),
    ("price", "@x"),
    ("points", "@y")
]

p.scatter(x='x', y='y', radius='circle_size', fill_alpha='fill_alpha',
          source=source)
#           fill_color=colors, fill_alpha=0.6,
#           line_color=None)


# output_file("color_scatter.html", title="color_scatter.py example")
output_notebook()
show(p)

Loading BokehJS ...